In [1]:
#comment out if using a local environment
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
cd "gdrive/My Drive/TopicModelling2/TopicModelling"

/content/gdrive/My Drive/TopicModelling2/TopicModelling


In [3]:
#download spacy's french language model and pyLDAvis for Topic Modelling visualization
!python -m spacy download fr_core_news_sm
!pip install pyLDAvis

In [4]:
import pandas as pd
import numpy as np
import warnings
from collections import Counter

#from utilsTM import corpusVisualizer, Output, deTokenize, compareCorpuses #Output is from ipywidgets
from utilsTM import *

from gensim.utils import simple_preprocess
from gensim.models.phrases import Phrases
from gensim.models.ldamodel import LdaModel, CoherenceModel
from gensim.corpora.dictionary import Dictionary

import pyLDAvis
import pyLDAvis.gensim #this is needed

import spacy

#import nltk

### Aim Of The Project
The aim of this notebook is to perform topic modelling on a corpus of company descriptions extracted from the [Commercial Register of Geneva](https://www.ge.ch/recherche-entreprises-registre-du-commerce-geneve).  
This could be used to analyze the economic landscape of the city and to build similarities from the topics embeddings.  
*(Please refer to the commercial register of Geneva for information on restrictions of use of the data)*

### Preprocessing the corpus
To get meaningful topics we need to remove words which are not relevant to categorize company descriptions.  
The steps are the following (the order can be changed):


1.   Remove punctations, lower case, accents ...
2.   Remove links, frequent unuseful information
3.   Tokenize
4.   Remove stop words, named entities
5.   Remove words other than noun, adjectif, verb or adverb
6.   Lemmatize
7.   Build bi-grams and tri-grams
8.   Remove frequent words, list of manually chosen words, verbs 

Not that the last step is specific to this task.



In [9]:
warnings.filterwarnings(action="ignore")

In [5]:
#read the data from a csv file. 
corpus0 = pd.read_csv("CommercialRegisterData.csv").Objet.to_list()

In [6]:
print("data size: ",len(corpus0))

data size:  9996


In [11]:
# we will use a small UI to vizualize the corpus more easily, as we go along the different preprocessing steps
Out0 = Output()
corpusVisualizer(corpus0,Out0)

In [12]:
#1. a lot of doc contain (cf. statuts pour but complet), (cf. but de l'établissement principal), ... at the end of the description. We remove them. 
corpus1 = [doc[:doc.find("(cf")] for doc in corpus0]

In [13]:
Out1 = Output()
corpusVisualizer(corpus1,Out1)

In [21]:
#2. Perform a number of preliminary preprocessing steps using gensim's simple_preprocess() (remove punctations, lowercase etc...).
corpus2 = [simple_preprocess(doc,deacc=False,max_len=30) for doc in corpus1]

In [22]:
#deTokenize is a utility function which converts the tokens back to a string.
Out2 = Output()
corpusVisualizer(deTokenize(corpus2),Out2)

In [49]:
#3. a. Remove stopwords, lemmatize (this might take a bit more time)
#load a spaCy french language model
nlp = spacy.load("fr_core_news_sm")
keepPos = ['NOUN', 'ADJ', 'VERB', 'ADV']
corpus3 = []
for doc2 in deTokenize(corpus2):
  doc2 = nlp(doc2) #perform text analysis on "doc" with the model from spacy.
  doc3 = [token.lemma_ for token in doc2 if not token.is_punct and not token.is_stop and (token.pos_ in keepPos) and not token.ent_type]
  corpus3.append(doc3)


In [25]:
Out3 = Output()
corpusVisualizer(deTokenize(corpus3),Out3)

In [74]:
with open(file="frenchVerbs.txt",mode="r") as f:
  FRENCHVERBS = list(f.readlines())
  FRENCHVERBS = [verb.replace("\n","").strip() for verb in FRENCHVERBS]
corpus4 = [[word for word in doc if word not in FRENCHVERBS] for doc in corpus3]

In [59]:
#4. construct bigrams tokens, using the Phrases model from gensim
#corpus4 = list(Phrases(sentences=corpus3,min_count=2)[corpus4])

In [ ]:
# detect trigrams
#corpus4 = list(Phrases(sentences=corpus4,min_count=2)[corpus4])

In [78]:
#b. Remove accents
corpus4 = [simple_preprocess(doc,deacc=True,max_len=30) for doc in deTokenize(corpus4)]

In [62]:
Out4 = Output()
corpusVisualizer(deTokenize(corpus4),Out4)

In [ ]:
cnt = Counter([word for doc in corpus4 for word in doc])
cnt.most_common(63)

In [81]:
#5. Remove pervasive words, which are present in most of the documents and are noise from a domain specific vocabulary (business) rather than discriminative features
#a. Take the most frequent word, manually set the threshold for the most common word number and keep some words which might be due to a frequent topic
cnt = Counter([word for doc in corpus4 for word in doc])
keep = ["financier","immobilier","courtage","batiment","materiel","importation_exportation","informatique","bien_immobilier","investissement","industriel","public"]
remove = [k for k,_ in cnt.most_common(63) if k not in keep]
manual = ["lfaie","but","pays","genre","aupres","groupe","lien","principalement","commercer","rapport","edition","geneve","premier","tiers"] #remove adverbes...
remove = remove + manual
#b. Automatically detect topics that are domain specific and not disciminative, by looking at the entropy of topic distributions

#remove words
corpus5 = [[word for word in doc if word not in remove] for doc in corpus4]

In [ ]:
#corpus4 = list(Phrases(sentences=corpus3,min_count=2)[corpus4])

In [79]:
#To compare corpuses
Out = Output()
compareCorpuses(corpus0,corpus4,Out)

In [66]:
#Final corpus remove empty docs
corpusFinal = [doc for doc in corpus5 if doc]

In [67]:
id2word = Dictionary(documents=corpusFinal)

In [68]:
corpusBow = [id2word.doc2bow(doc) for doc in corpusFinal]

In [43]:
#without removing frequent words, coherence 0.58, 15 topics

In [71]:
ntopics = 17
rs = 100
model = LdaModel(corpus=corpusBow,num_topics=ntopics,id2word=id2word,chunksize=2000,
                 passes=20,minimum_probability=0.0001,random_state=rs,per_word_topics=False)
coherence = CoherenceModel(model=model,texts=corpusFinal,dictionary=id2word).get_coherence()
print("Average correlation between multi-words in topics",round(coherence,4))

Average correlation between multi-words in topics 0.4654


In [72]:
#80 not so bad,10 not bad, 15 0.5 !!! #sometimes bigrams are confusing the thign
#model.save(fname="model80topics")

In [73]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model, corpusBow, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.278226 -0.168001       1        1  10.610544
11     0.122913  0.217816       2        1   9.803053
1      0.298288 -0.026673       3        1   8.072852
16     0.023333 -0.175388       4        1   7.151967
2      0.006088  0.155953       5        1   7.132213
13     0.020243  0.169216       6        1   6.067003
6     -0.055993 -0.122688       7        1   5.594629
4     -0.028868  0.094360       8        1   5.137751
0     -0.054810 -0.001833       9        1   4.931319
8     -0.121415 -0.148713      10        1   4.897367
3     -0.063524  0.107248      11        1   4.807278
5     -0.082013 -0.022504      12        1   4.561824
12    -0.050358 -0.071654      13        1   4.504440
9     -0.075503 -0.034222      14        1   4.458134
7     -0.019149  0.067848      15        1   4.301833
15    -0.116107 -0.060389      16        1   4.101625
14    -0.081350  0.019624      17        1   3.866167, topic_info=                 Term         Freq        Total Category  logprob  loglift
11         immobilier  1118.000000  1118.000000  Default  30.0000  30.0000
128         financier   937.000000   937.000000  Default  29.0000  29.0000
121          batiment   346.000000   346.000000  Default  28.0000  28.0000
195      informatique   432.000000   432.000000  Default  27.0000  27.0000
6            courtage   438.000000   438.000000  Default  26.0000  26.0000
...               ...          ...          ...      ...      ...      ...
779           affaire    21.020964   119.051444  Topic17  -4.8755   1.5189
146   direct_indirect    14.254564    37.631586  Topic17  -5.2639   2.2821
989        accessoire    15.919117   235.285648  Topic17  -5.1535   0.5596
425           boisson    14.209615    73.178944  Topic17  -5.2671   1.6139
1488      association    13.825919   114.652627  Topic17  -5.2944   1.1375

[885 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
805       2  0.256638       acces
805       5  0.718587       acces
989       4  0.119004  accessoire
989       7  0.480267  accessoire
989       9  0.191257  accessoire
...     ...       ...         ...
226      16  0.984948         vin
2006     12  0.962179      visuel
6010     17  0.927566    viticole
549       8  0.978463     voiture
503      11  0.955989         vol

[1411 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 12, 2, 17, 3, 14, 7, 5, 1, 9, 4, 6, 13, 10, 8, 16, 15])

In [ ]:
from gensim import similarities
index = similarities.MatrixSimilarity(model[corpusBow])

In [ ]:
sims = index[vec_lsi]

In [ ]:
vec_lsi

[(0, 0.02222222),
 (1, 0.02222222),
 (2, 0.02222222),
 (3, 0.02222222),
 (4, 0.02222222),
 (5, 0.02222225),
 (6, 0.02222222),
 (7, 0.02222222),
 (8, 0.02222222),
 (9, 0.02222222),
 (10, 0.02222222),
 (11, 0.02222222),
 (12, 0.02222222),
 (13, 0.3555555),
 (14, 0.3555556)]

In [ ]:
sorted(enumerate(sims), key=lambda item: -item[1])

In [ ]:
doc = "horlogerie metal precieux"
vec_bow = id2word.doc2bow(doc.lower().split())
vec_lsi = model[vec_bow]  # convert the query to LSI space
sims = index[vec_lsi]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
for i, s in sims[:200]:
    print(i,s, corpus0[i])